In [13]:
import gzip
import collections
from sklearn import linear_model
from sklearn import metrics
import random
import numpy as np
import string
import math
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from collections import defaultdict

In [2]:
def readGz(path):
    for l in gzip.open(path, 'rt', encoding='utf-8'):
        if 'null' not in l:
            yield eval(l)

In [3]:
all_cloth = list(readGz("renttherunway_final_data.json.gz"))

In [27]:
all_cloth[0]

{'fit': 'fit',
 'user_id': '420272',
 'bust size': '34d',
 'item_id': '2260466',
 'weight': '137lbs',
 'rating': '10',
 'rented for': 'vacation',
 'review_text': "An adorable romper! Belt and zipper were a little hard to navigate in a full day of wear/bathroom use, but that's to be expected. Wish it had pockets, but other than that-- absolutely perfect! I got a million compliments.",
 'body type': 'hourglass',
 'review_summary': 'So many compliments!',
 'category': 'romper',
 'height': '5\' 8"',
 'size': 14,
 'age': '28',
 'review_date': 'April 20, 2016'}

In [5]:
len(all_cloth)

192462

In [31]:
for item in all_cloth:
    keys_with_spaces = list(item.keys())  # Extracting keys to modify while iterating
    for key in keys_with_spaces:
        if ' ' in key:
            new_key = key.replace(' ', '_')  # Replacing spaces with underscores
            item[new_key] = item.pop(key)  # Replace the key in the dictionary

In [32]:
all_cloth[0]

{'fit': 'fit',
 'user_id': '420272',
 'item_id': '2260466',
 'weight': '137lbs',
 'rating': '10',
 'review_text': "An adorable romper! Belt and zipper were a little hard to navigate in a full day of wear/bathroom use, but that's to be expected. Wish it had pockets, but other than that-- absolutely perfect! I got a million compliments.",
 'review_summary': 'So many compliments!',
 'category': 'romper',
 'height': '5\' 8"',
 'size': 14,
 'age': '28',
 'review_date': 'April 20, 2016',
 'bust_size': '34d',
 'rented_for': 'vacation',
 'body_type': 'hourglass'}

In [33]:
#Data cleaning:
#1: We noticed that some clothing data between user and item is incomplete
cloths_key_len = []
for item in all_cloth:
    cloths_key_len.append(len(item.keys()))


In [34]:
#Incomplete clothing data for the item if the item has less than the standard 15 variables to describe it. 
unique_values = list(set(cloths_key_len))
print(unique_values)

[12, 13, 14, 15]


In [35]:
#Cleaned Dataset 
cloth_cleaned = []
for item in all_cloth:
    if len(item.keys()) == 15:
        cloth_cleaned.append(item)
len(cloth_cleaned)

146381

In [36]:
cloth_cleaned[0]

{'fit': 'fit',
 'user_id': '420272',
 'item_id': '2260466',
 'weight': '137lbs',
 'rating': '10',
 'review_text': "An adorable romper! Belt and zipper were a little hard to navigate in a full day of wear/bathroom use, but that's to be expected. Wish it had pockets, but other than that-- absolutely perfect! I got a million compliments.",
 'review_summary': 'So many compliments!',
 'category': 'romper',
 'height': '5\' 8"',
 'size': 14,
 'age': '28',
 'review_date': 'April 20, 2016',
 'bust_size': '34d',
 'rented_for': 'vacation',
 'body_type': 'hourglass'}

In [37]:
#User item pair for recommendation:
user_dict = defaultdict(list)
for item in cloth_cleaned:
    user_dict[item['user_id']].append(item['item_id'])

In [49]:
#Different occasions of clothes
cloth_occasions = []
for item in cloth_cleaned:
    cloth_occasions.append(item['rented_for'])

In [52]:
unique_occasions = list(set(cloth_occasions))
print(unique_occasions)

['vacation', 'other', 'formal affair', 'wedding', 'date', 'party', 'everyday', 'party: cocktail', 'work']


In [53]:
len(unique_occasions)

9